### Importing the needed libraries

In [1]:
import numpy as np
import cv2
import albumentations as A
import os
import pandas as pd
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from scipy.optimize import minimize
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import matplotlib.pyplot as plt
from matplotlib import style

### Creating the transformer for data augmentation

In [15]:
transform = A.Compose([
    A.HorizontalFlip(p=0.5),
    A.RandomRotate90(p=0.5),
    A.RandomBrightnessContrast(p=0.2),
    A.ShiftScaleRotate(shift_limit=0.05, scale_limit=0.05, rotate_limit=15, p=0.5),
])

### Augmenting the data for better predictions

In [22]:
augmentation_path = './Data/Training/no_tumor'
output = './Data/Training/augmented_no_tumor'
for img in os.listdir(augmentation_path):
    image = cv2.imread(os.path.join(augmentation_path, img))
    augmented = transform(image=image)['image']
    output_img_path = os.path.join(output, f"{img.split('.')[0]}_aug.jpg")
    cv2.imwrite(output_img_path, augmented)
    cv2.imwrite(output + '/' + img, image)

### Creating the train and test labels and data

In [2]:
train_data = []
train_labels = []
test_data = []
test_labels = []

### Scaling the data and splitting it and resizing it

In [3]:
for dir in os.listdir("./Data"):
    path = "./Data/" + dir
    for dir2 in os.listdir(path):
        path2 = path + "/" + dir2
        for img in os.listdir(path2):
            image = cv2.imread(path2 + "/" + img)
            image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
            image = cv2.resize(image, (28, 28))
            image = image.flatten() / 255.0
            if dir == "Testing":
                test_data.append(image)
                test_labels.append(dir2)
            else:
                train_data.append(image)
                train_labels.append(dir2)

### Convert the data type from `list` to `numpy array`

In [4]:
train_data = np.array(train_data)
train_labels = np.array(train_labels)
test_data = np.array(test_data)
test_labels = np.array(test_labels)

In [5]:
print(len(train_data), len(train_labels) , len(test_data), len(test_labels))

3265 3265 394 394


In [6]:
xtrain, xtest, ytrain, ytest = train_test_split(train_data, train_labels, test_size=0.2, random_state=42)

### Convert our labels to number for better handling

In [7]:
le = preprocessing.LabelEncoder()

le.fit(train_labels)
train_labels = le.transform(train_labels)
le.fit(test_labels)
test_labels = le.transform(test_labels)

In [8]:
train_data.shape, xtrain.shape

((3265, 784), (2612, 784))

### Support Vector Machine From Scratch